## Tarea 5

Escuela de Ingeniería Electrónica<br>
EL5857 Aprendizaje Automático<br>
I Semestre 2023

In [1]:
import os
import pickle
import torch
import tkinter as tk
import torch.nn as nn
import torch.nn.functional as F

# Some version magic here
from PIL import Image,ImageDraw
from torchvision import transforms

try:
    from PIL import ImageOps, Resampling
    resampling_filter = Resampling.LANCZOS
except ImportError:
    resampling_filter = Image.LANCZOS

import numpy as np

In [2]:
class DigitPainter(tk.Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        
        self.lastX, self.lastY = 0, 0        
        self.pencil=55        
        self.imgside=448
        
        self.h=int((self.pencil-1)/2)
        self.create_widgets()
        
        
    def create_widgets(self):
        self.master.geometry('448x512')  # 448=28x16
        self.master.title("Drawing window")

        self.img = Image.new(mode='L',size=(self.imgside,self.imgside),color=(0)) # 'L' means 8-bits        
        self.painter=ImageDraw.Draw(self.img)
        self.canvas= tk.Canvas(self.master,width=self.imgside,height=self.imgside,bg="black")
               
        self.canvas.bind( "<Button-1>", self.get_x_and_y )
        self.canvas.bind( "<B1-Motion>", self.draw_box )
        self.canvas.pack() #expand = YES, fill = BOTH)
        
        # Botones y acciones
        self.frame = tk.Frame(self.master)
        self.frame.pack()

        self.resetButton=tk.Button(self.frame,text="Reset",command= self.erase_canvas) 
        self.resetButton.pack(side= tk.LEFT)
        
        self.svmButton=tk.Button(self.frame,text="SVM",command= self.predictSVM)
        self.svmButton.pack(side= tk.RIGHT)

        self.annButton=tk.Button(self.frame,text="ANN",command= self.predictANN)
        self.annButton.pack(side= tk.RIGHT)

        self.knnButton=tk.Button(self.frame,text="KNN",command= self.predictKNN)
        self.knnButton.pack(side= tk.RIGHT)

        self.rdfButton=tk.Button(self.frame,text="RDF",command= self.predictRDF)
        self.rdfButton.pack(side= tk.RIGHT)
        
        
    # Called when the button is pressed the first time
    def get_x_and_y(self,event):
        self.lastX,self.lastY = event.x, event.y

        self.svmButton['state']=tk.NORMAL
        self.annButton['state']=tk.NORMAL
        self.knnButton['state']=tk.NORMAL
        self.rdfButton['state']=tk.NORMAL

        self.resetButton['state']=tk.NORMAL
        
    # Called on a drag event
    def draw_box(self,event):
        
        self.canvas.create_line((self.lastX,self.lastY,event.x,event.y),fill='white',width=self.pencil)
        self.canvas.create_oval((event.x-self.h,event.y-self.h,event.x+self.h,event.y+self.h),
                                fill='white',outline='white')
        
        self.painter.line([(self.lastX,self.lastY),(event.x,event.y)],fill='white',width=self.pencil)
        self.painter.ellipse([(event.x-self.h,event.y-self.h),(event.x+self.h,event.y+self.h)],
                               fill='white',outline='white')        
        
        self.lastX,self.lastY=event.x,event.y

        
    #Encargado de borrar todo (para bóton)
    def erase_canvas(self):
    
        self.canvas.delete('all')  
        
        self.svmButton['state']=tk.DISABLED
        self.annButton['state']=tk.DISABLED
        self.knnButton['state']=tk.DISABLED
        self.rdfButton['state']=tk.DISABLED
        
        self.resetButton['state']=tk.DISABLED
    
        self.painter.rectangle([0,0,self.imgside,self.imgside],fill='black',outline='black')
        
    def getMNISTImage(self):
        return np.array(self.img.resize((28,28),resampling_filter))
        
    # ---------------------------
    # Your code below this point
        
    def loadPredictors(self):
        
        print("Loading predictors...")
        
        # ADD YOUR CODE HERE TO LOAD ALL FOUR PREDICTORS        

    def predictSVM(self):
        print("Predict SVM called")
        img=self.getMNISTImage()
        
        # ADD YOUR CODE HERE TO PREDICT WITH SVM
        # Cargar el modelo desde el archivo
        with open('C:/Users/Gollo/OneDrive/Desktop/svm_model.pkl', 'rb') as file:
            svm_model = pickle.load(file)
            
        # Normalizar la imagen y aplanarla para que sea un vector de características
        img = img.reshape(1, -1) / 255.
        
        # Predecir la clase de la imagen con el modelo SVM
        prediction = svm_model.predict(img)
        print("Predicted class:", prediction[0])

    def predictANN(self):
        print("Predict ANN called")
        img=self.getMNISTImage()

        # Definición de la clase Net
        class Net(nn.Module):
            def __init__(self):
                super(Net, self).__init__()
                # Capas de la red
                self.fc1 = nn.Linear(28 * 28, 512)
                self.fc2 = nn.Linear(512, 10)

            def forward(self, x):
                x = x.view(-1, 28 * 28)
                x = F.relu(self.fc1(x))
                x = self.fc2(x)
                return F.log_softmax(x, dim=1)

        # Cargar el estado del modelo
        state_dict = torch.load('C:/Users/Gollo/OneDrive/Desktop/model.pth')

        # Crear una instancia del modelo y cargar el estado
        model = Net()
        model.load_state_dict(state_dict)

        # Transformaciones para preprocesar la imagen
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

        
        # Convertir la imagen a una imagen PIL
        img = Image.fromarray(img)
        
        # Preprocesar la imagen
        img = transform(img)
        

        # Hacer la predicción
        output = model(img)
        prediction = output.argmax(dim=1).item()
        print(f'Predicted class: {prediction}')

        
    def predictKNN(self):
        print("Predict KNN called")
        img=self.getMNISTImage()
        
        # ADD YOUR CODE HERE TO PREDICT WITH KNN
        # Cargar el modelo desde el archivo
        with open('C:/Users/Gollo/OneDrive/Desktop/knn_model.pkl', 'rb') as file:
            knn_model = pickle.load(file)
            
        # Normalizar la imagen y aplanarla para que sea un vector de características
        img = img.reshape(1, -1) / 255.
        
        # Predecir la clase de la imagen con el modelo KNN
        prediction = knn_model.predict(img)
        print("Predicted class:", prediction[0])
        
    def predictRDF(self):
        print("Predict RDF called")
        img=self.getMNISTImage()
        
        # ADD YOUR CODE HERE TO PREDICT WITH RDF
        # Cargar el modelo desde el archivo
        with open('C:/Users/Gollo/OneDrive/Desktop/rf_model.pkl', 'rb') as file:
            rf_model = pickle.load(file)
            
        # Normalizar la imagen y aplanarla para que sea un vector de características
        img = img.reshape(1, -1) / 255.
        
        # Predecir la clase de la imagen con el modelo RDF
        prediction = rf_model.predict(img)
        print("Predicted class:", prediction[0])
            

In [3]:
root = tk.Tk()
painter=DigitPainter(master=root)
painter.mainloop()

Predict RDF called
Predicted class: 2
Predict KNN called
Predicted class: 5
Predict SVM called
Predicted class: 4
Predict ANN called
Predicted class: 4
Predict RDF called
Predicted class: 3
Predict KNN called
Predicted class: 3
Predict SVM called
Predicted class: 3
Predict ANN called
Predicted class: 3
Predict ANN called
Predicted class: 1
Predict RDF called
Predicted class: 8
Predict KNN called
Predicted class: 1
Predict SVM called
Predicted class: 1
Predict RDF called
Predicted class: 9
Predict KNN called
Predicted class: 9
Predict SVM called
Predicted class: 9
Predict ANN called
Predicted class: 9
Predict RDF called
Predicted class: 1
Predict KNN called
Predicted class: 1
Predict SVM called
Predicted class: 1
Predict ANN called
Predicted class: 1
Predict RDF called
Predicted class: 2
Predict KNN called
Predicted class: 1
Predict SVM called
Predicted class: 2
Predict ANN called
Predicted class: 2
Predict RDF called
Predicted class: 2
Predict KNN called
Predicted class: 1
Predict SVM 